# 基于卷积神经网络的手写英文字母识别系统研究

## 准备数据集及数据预处理

### 将下载的数据集按类重命名

In [1]:
"""""
import os
import string

# 定义源目录路径
source_dir = "EnglishImg/EnglishImg/English/Img/GoodImg/Bmp"

# 生成目标文件夹名称列表
target_folders = list(string.digits) + list(string.ascii_uppercase) + [f"{char}_" for char in string.ascii_lowercase]

# 获取源目录下的所有文件夹名称
source_folders = sorted([f for f in os.listdir(source_dir) if os.path.isdir(os.path.join(source_dir, f))])

# 确保源文件夹数量与目标文件夹数量一致
if len(source_folders) != len(target_folders):
    raise ValueError("源文件夹数量与目标文件夹数量不一致")

# 重命名文件夹
for source_folder, target_folder in zip(source_folders, target_folders):
    source_path = os.path.join(source_dir, source_folder)
    target_path = os.path.join(source_dir, target_folder)

    try:
        os.rename(source_path, target_path)
        print(f"重命名: {source_path} -> {target_path}")
    except FileExistsError:
        print(f"目标文件夹 {target_path} 已存在，跳过重命名 {source_path}")
    except Exception as e:
        print(f"重命名 {source_path} 到 {target_path} 时出错: {e}")
"""""




'""\nimport os\nimport string\n\n# 定义源目录路径\nsource_dir = "EnglishImg/EnglishImg/English/Img/GoodImg/Bmp"\n\n# 生成目标文件夹名称列表\ntarget_folders = list(string.digits) + list(string.ascii_uppercase) + [f"{char}_" for char in string.ascii_lowercase]\n\n# 获取源目录下的所有文件夹名称\nsource_folders = sorted([f for f in os.listdir(source_dir) if os.path.isdir(os.path.join(source_dir, f))])\n\n# 确保源文件夹数量与目标文件夹数量一致\nif len(source_folders) != len(target_folders):\n    raise ValueError("源文件夹数量与目标文件夹数量不一致")\n\n# 重命名文件夹\nfor source_folder, target_folder in zip(source_folders, target_folders):\n    source_path = os.path.join(source_dir, source_folder)\n    target_path = os.path.join(source_dir, target_folder)\n\n    try:\n        os.rename(source_path, target_path)\n        print(f"重命名: {source_path} -> {target_path}")\n    except FileExistsError:\n        print(f"目标文件夹 {target_path} 已存在，跳过重命名 {source_path}")\n    except Exception as e:\n        print(f"重命名 {source_path} 到 {target_path} 时出错: {e}")\n'

### 划分训练集和测试集

In [4]:
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torchvision.transforms as T
from myCnn.baseline.LeNet5 import LeNet5
from myCnn.train_model import train_model
from myCnn.utils import split_dataset
from myCnn.baseline.resnet18 import resnet18
from myCnn.baseline.mobilenet_v2 import mobilenet_v2
from myCnn.baseline.svm_model import train_svm, flatten_images
EPOCH = 100
BATCH_SIZE = 128
LR = 1e-4

ModuleNotFoundError: No module named 'sklearn'

In [ ]:
# 定义数据预处理变换
transform = T.Compose([
    T.Resize((32, 32)),
    # T.Grayscale(num_output_channels=1),
    T.ToTensor(),
    # 如果需要标准化，可以取消注释以下行
    # T.Normalize([0.5], [0.5])
])



In [ ]:
# 使用函数划分数据集
train_loader, val_loader, test_loader, full_dataset = split_dataset(
    root_dir="EnglishImg/EnglishImg/English/Img/GoodImg/Bmp",
    transform=transform,
    train_ratio=0.7,
    val_ratio=0.15,
    test_ratio=0.15,
    batch_size=BATCH_SIZE,
    shuffle=True,
    random_seed=42
)

# 打印数据集大小
print(f"训练集大小: {len(train_loader.dataset)}")
print(f"验证集大小: {len(val_loader.dataset)}")
print(f"测试集大小: {len(test_loader.dataset)}")

# 获取类别数量
label_num = len(full_dataset.class_to_idx)

# 数据可视化
to_img = T.ToPILImage()
a = to_img(train_loader.dataset[0][0])  # size=[1, 28, 28]
plt.imshow(a)
plt.axis('off')
plt.show()


# 获取类别数量
label_num = len(full_dataset.class_to_idx)

In [ ]:

# # 获取展平后的训练集和验证集
# X_train, y_train = flatten_images(train_loader)
# X_val, y_val = flatten_images(val_loader)

In [ ]:

# # 训练SVM模型
# svm_clf, train_acc, val_acc = train_svm(X_train, y_train, X_val, y_val)

In [ ]:

# 初始化模型
cnn = mobilenet_v2(pretrained=True)
print(cnn)

# 检查 CUDA 是否可用
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
cnn.to(device)  # 将模型迁移到 GPU

# 定义优化器
optimizer = torch.optim.Adam(cnn.parameters(), lr=LR)
# 定义损失函数
loss_func = nn.CrossEntropyLoss()

# 训练模型
cnn = train_model(cnn, train_loader, val_loader, loss_func, optimizer, num_epochs=EPOCH)

